In [119]:
using Concorde
using LinearAlgebra

In [168]:
using Printf


function _assign!(T, M)
    W = eltype(T)

    T .= zero(W)
    _, indices = findmax(M, dims=2)
    T[indices] .= one(W)
end


function _saddle_tsp!(T, Q, A, Λ, Σ, P; maxit)
    n = size(T, 1)

    lT = copy(T)
    for _ in 1:maxit
        _assign!(Q, P' * T' * A - Λ' - Σ' + 2 * Σ' .* T')
        _assign!(T, A * Q' * T' + Λ - Σ + 2 * Σ .* Q')

        if all(lT .== T)
            break
        end

        lT .= T
    end
end


function _objective_tsp(T, A, P)
    sum((T * P * T') .* A)
end


function atsp(A; lr=1.0, lrdecay=sqrt(.999), decay=.999, maxit)
    n = size(A, 1)

    T = zeros(n, n)
    Q = zeros(n, n)
    Λ = zeros(n, n)
    Σ = zeros(n, n)

    Tbest = copy(T)
    best = -Inf

    P = [zeros(n - 1) I(n - 1); 1 zeros(n - 1)']

    for it in 1:maxit
        _saddle_tsp!(T, Q, A, Λ, Σ, P, maxit=1)

        ∇ = Q' - T
        norm = sum(abs.(∇))

        if norm <= 0
            obj = _objective_tsp(T, A, P)
            if obj > best
                Tbest, best = copy(T), obj
                @printf("best = %5.3f\tlr = %5.3f\tit = %d\n",
                    best, lr, it)
                flush(stdout)
            end

            Σ *= decay
            Λ *= decay
            lr *= lrdecay
        end

        Σ += lr * abs.(∇)
        Λ += lr * ∇
    end

    Tbest
end

atsp (generic function with 1 method)

In [220]:
n = 500
A = rand(Int, n, n) .% 2
A = min.(A, A')
A = ifelse.(A .< 0, 0, A)
A[diagind(A)] .= 0
A

500×500 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  0  1  1  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  1  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0   

In [221]:
_, score = solve_tsp(-A)
-score

500

In [223]:
M = Float64.(A)
T = atsp(M, lr=10.0, maxit=300000);
_objective_tsp(T, M)

best = 133.000	lr = 10.000	it = 3615


LoadError: InterruptException: